In [1]:
import gym
import gym_cellular_automata as gymca


In [2]:
from Solver_Firefighter.reward_wrappers import NumTreesRewardStepOne
import numpy as np

SQUARE_SHAPE = 10
T_MOVE = 0.025
T_SHOOT = 0.1
T_ANY = 0.025
POS_BULL = [SQUARE_SHAPE-1, SQUARE_SHAPE-1]
BULL_POS = f'{SQUARE_SHAPE-1}.{SQUARE_SHAPE-1}'
POS_FIRE = (5,0)


ProtoEnv = gymca.prototypes[1]
env = ProtoEnv(nrows=SQUARE_SHAPE,
               ncols=SQUARE_SHAPE,
               pos_bull=POS_BULL,
               pos_fire=POS_FIRE,
               t_move=T_MOVE,
               t_shoot=T_SHOOT,
               t_any=T_ANY)

env.reset()
env.render()
obs = env.reset()
env = NumTreesRewardStepOne(env)
#env.render()
total_reward = 0.0
done = False
step = 0
threshold = 30

env.render()
print("Total trees: ", np.sum(np.where(env.grid == 3, 1, 0)))
total_loss = 0
#env.observation_space.sample()

done = True


while not done:
    action = env.action_space.sample()  # Your agent goes here!
    obs, reward, done, info = env.step([4,1])

    env.render()
    if reward != 0:
        total_loss += reward
        print("reward", reward, "total loss", total_loss)


env.render()
print("Total trees: ", np.sum(np.where(env.grid == 3, 1, 0)))


Total trees:  38
Total trees:  38


In [3]:
from stable_baselines3 import A2C, SAC, PPO, TD3, DQN
import os


In [4]:
import os

save_dir = "avatar-graph_models"
os.makedirs(save_dir, exist_ok=True)


In [5]:
# We going to save the total burnt nodes.
RL_objective = {}

In [7]:
# For loop over all models
import imageio
from stock_envs import make_env

ESSAYS = 2


MODEL = PPO
NAME_MODEL = 'PPO'
TOTAL_PROCS = 8
TRAIN_STEPS = 1000#6e6
STEPS_TRESH = int(1000)
ENV_SEED = 100
NAME_REWARD = 'NumTreesRewardStepOne'
OBS_WRAP = 'ObservationOneHotWrapper_dict_box'
POLICY_TYPE = 'MultiInputPolicy'


move_values = {
    0: {"T_MOVE": 0.025, "T_SHOOT": 0.1, "T_ANY": 0.025},
    1: {"T_MOVE": 0.05, "T_SHOOT": 0.3, "T_ANY": 0.05},
    2: {"T_MOVE": 0.1, "T_SHOOT": 0.5, "T_ANY": 0.05},
    3: {"T_MOVE": 0.5, "T_SHOOT": 0.5, "T_ANY": 0.05},
    4: {"T_MOVE": 0.1, "T_SHOOT": 0.15, "T_ANY": 0.03, "D": [4]},
    5: {"T_MOVE": 0.2, "T_SHOOT": 0.025, "T_ANY": 0.025, "D": [5]},
    6: {"T_MOVE": 0.3, "T_SHOOT": 0.1, "T_ANY": 0.025, "D": [3]},
    7: {"T_MOVE": 0.4, "T_SHOOT": 0.05, "T_ANY": 0.05, "D": [3]},
}

for instance_value in range(8):
    T_MOVE = move_values[instance_value]["T_MOVE"]
    T_SHOOT = move_values[instance_value]["T_SHOOT"]
    T_ANY = move_values[instance_value]["T_ANY"]

    for POS_FIRE in [(5, 0), (4, 5)]:

        parameters = {
            'nrows': SQUARE_SHAPE,
            'ncols': SQUARE_SHAPE,
            'pos_bull': POS_BULL,
            'pos_fire': POS_FIRE,
            't_move': T_MOVE,
            't_shoot': T_SHOOT,
            't_any': T_ANY,
            #'model': MODEL,
            'name_model': NAME_MODEL,
            'total_proces': TOTAL_PROCS,
            'steps_threshold': STEPS_TRESH,
            'env_seed': ENV_SEED,
            'name_reward': NAME_REWARD,
            'obs_wrapper': OBS_WRAP,
            'policy_type': POLICY_TYPE#"MlpPolicy"
        }

        run_name = f"avatar-fire-master-{instance_value}-{'a' if POS_FIRE == (5, 0) else 'b'}"
        print(run_name)
        env_fn = make_env(ProtoEnv, 0, parameters)  # Get the environment creation function
        env = env_fn()  # Create the environment instance
        loaded_model = PPO.load(os.path.join(save_dir, run_name + ".zip"), env=env)  # Load the trained model

        obs = env.reset()
        #print(obs)


        for essay in range(ESSAYS):
            total_reward = 0.0
            done = False
            step = 0
            threshold = 15
            env.reset()
            #env.render()

            images = []
            obs = env.reset()
            img = env.render()

            while not done: # and step < threshold:
                #print(step)
                images.append(img)
                obs_reshaped = np.expand_dims(np.expand_dims(obs, axis=0), axis=0)  # Reshape the observation
                action = loaded_model.predict(obs, deterministic=True)[0]
                obs, reward, done, info = env.step(action)
                total_reward += reward
                step += 1
                env.render()
                img = env.render()

            print('Done', done, step)
            print('Total_reward', total_reward)

            burt_nodes = np.sum(np.where(env.grid == 25, 1, 0))

            RL_objective[run_name] = {
                'total_reward': total_reward,
                'burt_nodes': burt_nodes,
                'step': step
            }

            save_dir_gif = f"gifs"
            os.makedirs(save_dir_gif, exist_ok=True)

            gif_name = save_dir_gif + f"/{run_name}" + f"-{str(essay)}"
            imageio.mimsave(f"{gif_name}.gif", images, fps=3)
            env.close()

# save the RL_objective
import pickle
with open('RL_objective.pkl', 'wb') as f:
    pickle.dump(RL_objective, f, pickle.HIGHEST_PROTOCOL)

avatar-fire-master-0-a


FileNotFoundError: [Errno 2] No such file or directory: 'avatar-graph_models/avatar-fire-master-0-a.zip.zip'

17.0

In [5]:
loaded_model = PPO.load(os.path.join(save_dir, run_name + ".zip"))  # Include the file extension when loading

# Check that the prediction is the same after loading (for the same observation)
#print("loaded", loaded_model.predict(obs, deterministic=True))


In [6]:
#loaded_model = PPO.load(save_dir + run_name)
# Check that the prediction is the same after loading (for the same observation)
#print("loaded", loaded_model.predict(obs, deterministic=True))

In [6]:
import imageio
from wrappers import ObservationOneHotWrapper
from reward_wrappers import NumTreesReward, NumTreesRewardStepOne
from stable_baselines3.common.vec_env import DummyVecEnv
from stock_envs import make_env

INSTANCE_VALUE = 2

MODEL = PPO
NAME_MODEL = 'PPO'
TOTAL_PROCS = 8
TRAIN_STEPS = 1000#6e6
STEPS_TRESH = int(1000)
ENV_SEED = 100
NAME_REWARD = 'NumTreesRewardStepOne'
OBS_WRAP = 'ObservationOneHotWrapper_dict_box'
POLICY_TYPE = 'MultiInputPolicy'

move_values = {
    0: {"T_MOVE": 0.025, "T_SHOOT": 0.1, "T_ANY": 0.025},
    1: {"T_MOVE": 0.05, "T_SHOOT": 0.3, "T_ANY": 0.05},
    2: {"T_MOVE": 0.1, "T_SHOOT": 0.5, "T_ANY": 0.05},
    3: {"T_MOVE": 0.5, "T_SHOOT": 0.5, "T_ANY": 0.05},
}

T_MOVE = move_values[INSTANCE_VALUE]["T_MOVE"]
T_SHOOT = move_values[INSTANCE_VALUE]["T_SHOOT"]
T_ANY = move_values[INSTANCE_VALUE]["T_ANY"]

parameters = {
    'nrows': SQUARE_SHAPE,
    'ncols': SQUARE_SHAPE,
    'pos_bull': POS_BULL,
    'pos_fire': POS_FIRE,
    't_move': T_MOVE,
    't_shoot': T_SHOOT,
    't_any': T_ANY,
    #'model': MODEL,
    'name_model': NAME_MODEL,
    'total_proces': TOTAL_PROCS,
    'steps_threshold': STEPS_TRESH,
    'env_seed': ENV_SEED,
    'name_reward': NAME_REWARD,
    'obs_wrapper': OBS_WRAP,
    'policy_type': POLICY_TYPE#"MlpPolicy"
}


In [7]:
env_fn = make_env(ProtoEnv, 0, parameters)  # Get the environment creation function
env = env_fn()  # Create the environment instance
loaded_model = PPO.load(os.path.join(save_dir, run_name + ".zip"), env=env)  # Load the trained model

obs = env.reset()
print(obs)
#obs = np.expand_dims(obs, axis=0)  # Expand the observation shape to (1, 1)
done = False
total_reward = 0

while not done:
    action, _ = loaded_model.predict(obs, deterministic=True)  # Get the action from the model
    obs, reward, done, _ = env.step(action)  # Take the action in the environment
    total_reward += reward
    env.render()

print('Total Reward:', total_reward)



Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
{'grid': array([[[0., 0., 0., 1., 0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 0., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 1., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1., 1., 1., 1., 0., 1.],
        [0., 1., 1., 1., 1., 1., 0., 0., 0., 1.],
        [0., 1., 1., 0., 0., 1., 1., 0., 0., 0.],
        [0., 0., 1., 1., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 1., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0.,

In [7]:
import imageio
from wrappers import ObservationOneHotWrapper
from reward_wrappers import NumTreesReward, NumTreesRewardStepOne
from stable_baselines3.common.vec_env import DummyVecEnv
from stock_envs import make_env



# Play
ESSAYS = 3
train_steps = 'k'


env_fn = make_env(ProtoEnv, 0, parameters)  # Get the environment creation function
env = env_fn()  # Create the environment instance
loaded_model = PPO.load(os.path.join(save_dir, run_name + ".zip"), env=env)  # Load the trained model

obs = env.reset()
print(obs)
#obs = np.expand_dims(obs, axis=0)  # Expand the observation shape to (1, 1)

# while not done:
#     action, _ = loaded_model.predict(obs, deterministic=True)  # Get the action from the model
#     obs, reward, done, _ = env.step(action)  # Take the action in the environment
#     total_reward += reward
#     env.render()
#
# print('Total Reward:', total_reward)



for essay in range(ESSAYS):
    total_reward = 0.0
    done = False
    step = 0
    threshold = 15
    env.reset()
    #env.render()

    images = []
    obs = env.reset()
    img = env.render()

    while not done: # and step < threshold:
        #print(step)
        images.append(img)
        obs_reshaped = np.expand_dims(np.expand_dims(obs, axis=0), axis=0)  # Reshape the observation
        action = loaded_model.predict(obs, deterministic=True)[0]
        obs, reward, done, info = env.step(action)
        total_reward += reward
        step += 1
        env.render()
        img = env.render()

    print('Done', done, step)
    print('Total_reward', total_reward)

    save_dir_gif = f"gifs"
    os.makedirs(save_dir_gif, exist_ok=True)

    gif_name = save_dir_gif + f"/{run_name}" + f"-{str(essay)}"
    imageio.mimsave(f"{gif_name}.gif", images, fps=3)


Wrapping the env in a DummyVecEnv.
{'grid': array([[[0., 0., 0., 1., 0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 0., 1., 1., 1., 1.],
        [0., 0., 0., 0., 1., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 1., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1., 1., 1., 1., 0., 1.],
        [0., 1., 1., 1., 1., 1., 0., 0., 0., 1.],
        [0., 1., 1., 0., 0., 1., 1., 0., 0., 0.],
        [0., 0., 1., 1., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 1., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
    

In [8]:
for i in range(1,10):
    print(i)


1
2
3
4
5
6
7
8
9
